<a href="https://colab.research.google.com/github/JanMarcelKezmann/Residual-Network-Architectures/blob/master/Inception_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import tensorflow as tf

from IPython.display import SVG
from matplotlib.pyplot import imshow
%matplotlib inline

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

import tensorflow.keras.backend as K
K.set_image_data_format("channels_last")
K.set_learning_phase(1)

from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Add, Dense, Dropout, Lambda, Flatten, Concatenate, Input, Conv2D, Activation, BatchNormalization, AveragePooling2D, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers, activations

In [0]:
def inception_block_a(X, block):
    name = "inc_block_a" + block

    # Branch 1
    branch1 = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch1", padding="same", kernel_initializer=glorot_uniform(seed=0))(X)

    # Branch 2
    X_padded = ZeroPadding2D((1, 1), name=name + "_zeros")(X)
    branch2 = AveragePooling2D((3, 3), strides=(1, 1), name=name + "_avgpool")(X_padded)
    branch2 = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch2", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch2)

    # Branch 3
    branch3 = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch3_0", padding="same", kernel_initializer=glorot_uniform(seed=0))(X)
    #   branch3_padded = ZeroPadding2D((1, 1))(branch3)
    branch3 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), name=name + "_branch3_1", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch3)

    # Branch 4
    branch4 = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch4_0", padding="same", kernel_initializer=glorot_uniform(seed=0))(X)
    #   branch4_padded = ZeroPadding2D((1, 1))(branch4)
    branch4 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), name=name + "_branch4_1", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
    branch4 = Conv2D(filters=96, kernel_size=(3, 3), strides=(1, 1), name=name + "_branch4_2", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)

    branches = [branch1, branch2, branch3, branch4]

    mixed = Concatenate(axis=3, name=name + "_concat")(branches)
    mixed = BatchNormalization(axis=3, name=name + "_bn")(mixed)
    mixed = Activation("relu", name=name + "_activate")(mixed)

    return mixed

In [0]:
def inception_block_b(X, block, n=7, strides_bool=False):
    if strides_bool:
        strides = 2
    else:
        strides = 1

    name = "inc_block_b" + block

    m = int(math.floor((n - 1) / 2))

    # Branch 1
    if strides_bool:
        branch1 = Conv2D(filters=192, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    else:
        branch1 = Conv2D(filters=192, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)

    # Branch 2
    X_padded = ZeroPadding2D((1, 1), name=name + "_zeros")(X)
    branch2 = AveragePooling2D((3, 3), strides=(1, 1), name=name + "_avgpool")(X_padded)
    if strides_bool:
        branch2 = Conv2D(filters=192, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch2", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch2)
    else:
        branch2 = Conv2D(filters=192, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch2", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch2)

    # Branch 3
    branch3 = Conv2D(filters=192, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch3_0", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    if strides_bool:
        branch3 = Conv2D(filters=192, kernel_size=(m, m), strides=(strides, strides), name=name + "_branch3_1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch3)
    else:
        #   branch3_padded = ZeroPadding2D((m, m))(branch3)
        branch3 = Conv2D(filters=192, kernel_size=(1, n), strides=(1, 1), name=name + "_branch3_1", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch3)
        branch3 = Conv2D(filters=192, kernel_size=(n, 1), strides=(1, 1), name=name + "_branch3_2", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch3)

    # Branch 4
    branch4 = Conv2D(filters=192, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch4_0", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    if strides_bool:
        branch4 = Conv2D(filters=192, kernel_size=(m, m), strides=(1, 1), name=name + "_branch4_1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch4)
        branch4 = Conv2D(filters=192, kernel_size=(m, m), strides=(strides, strides), name=name + "_branch4_3", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
    else:
        #   branch4_padded = ZeroPadding((m, m))(branch4)
        branch4 = Conv2D(filters=192, kernel_size=(1, n), strides=(1, 1), name=name + "_branch4_1", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
        branch4 = Conv2D(filters=192, kernel_size=(n, 1), strides=(1, 1), name=name + "_branch4_2", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
        #   branch4_padded = ZeroPadding((m, m))(branch4)
        branch4 = Conv2D(filters=192, kernel_size=(1, n), strides=(1, 1), name=name + "_branch4_3", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
        branch4 = Conv2D(filters=192, kernel_size=(n, 1), strides=(1, 1), name=name + "_branch4_4", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)

    branches = [branch1, branch2, branch3, branch4]

    mixed = Concatenate(axis=3, name=name + "_concat")(branches)
    mixed = BatchNormalization(axis=3, name=name + "_bn")(mixed)
    mixed = Activation("relu", name=name + "_activate")(mixed)

    return mixed

In [0]:
def inception_block_c(X, block, strides_bool=False):
    if strides_bool:
        strides = 2
    else:
        strides = 1

    name = "inc_block_c" + block

    # Branch 1
    if strides_bool:
        branch1 = Conv2D(filters=320, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    else:
        branch1 = Conv2D(filters=320, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)

    # Branch 2
    X_padded = ZeroPadding2D((1, 1), name=name + "_zeros")(X)
    branch2 = AveragePooling2D((3, 3), strides=(1, 1), name=name + "_avgpool")(X_padded)
    if strides_bool:
        branch2 = Conv2D(filters=192, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch2", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch2)
    else:
        branch2 = Conv2D(filters=192, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch2", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch2)

    # Branch 3
    branch3 = Conv2D(filters=384, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch3_0", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    if strides_bool:
        branch3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch3_1", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch3)
    else:
        #   branch3_padded = ZeroPadding2D((1, 1))(branch3)
        branch3a = Conv2D(filters=384, kernel_size=(1, 3), strides=(1, 1), name=name + "_branch3_1a", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch3)
        branch3b = Conv2D(filters=384, kernel_size=(3, 1), strides=(1, 1), name=name + "_branch3_1b", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch3)

    # Branch 4
    branch4 = Conv2D(filters=384, kernel_size=(1, 1), strides=(1, 1), name=name + "_branch4_0", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    #   branch4_padded = ZeroPadding2D((1, 1))(branch4)
    branch4 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), name=name + "_branch4_1", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
    if strides_bool:
        branch4 = Conv2D(filters=384, kernel_size=(3, 3), strides=(strides, strides), name=name + "_branch4_2", padding="valid", kernel_initializer=glorot_uniform(seed=0))(branch4)
    else:
        #   branch4_padded = ZeroPadding2D((1, 1))(branch4)
        branch4a = Conv2D(filters=384, kernel_size=(1, 3), strides=(1, 1), name=name + "_branch4_2a", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)
        branch4b = Conv2D(filters=384, kernel_size=(3, 1), strides=(1, 1), name=name + "_branch4_2b", padding="same", kernel_initializer=glorot_uniform(seed=0))(branch4)

    if strides_bool:
        branches = [branch1, branch2, branch3, branch4]
    else:
        branches = [branch1, branch2, branch3a, branch3b, branch4a, branch4b]

    mixed = Concatenate(axis=3, name=name + "_concat")(branches)
    mixed = BatchNormalization(axis=3, name=name + "_bn")(mixed)
    mixed = Activation("relu", name=name + "_activate")(mixed)

    return mixed  

In [0]:
def inception_v3(input_shape=(299, 299, 3), classes=1000):
    # Define Input as tensor with input_shape as shape
    X_input = Input(input_shape)

    #   ZeroPadding2D((3, 3))(X_input) (adds 3 zeros on each side)
    #   X = ZeroPadding2D((3, 3))(X_input)
    #   print(X.shape) # should be (n + 3, n + 3, 3)

    # Stage 1
    X = Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), name="conv1a", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), name="conv1b", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), name="conv1c", padding="same", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="conv1_bn1")(X)
    X = Activation("relu")(X)

    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = Conv2D(filters=80, kernel_size=(3, 3), strides=(1, 1), name="conv1d", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(filters=192, kernel_size=(3, 3), strides=(2, 2), name="conv1e", padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), name="conv1f", padding="same", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="conv1_bn2")(X)
    X = Activation("relu")(X)

    # Inception A Block
    X = inception_block_a(X, block="_round1")
    X = inception_block_a(X, block="_round2")
    X = inception_block_a(X, block="_round3")

    # Inception B Block
    X = inception_block_b(X, block="_round1", n=7, strides_bool=True)
    X = inception_block_b(X, block="_round2", n=7)
    X = inception_block_b(X, block="_round3", n=7)
    X = inception_block_b(X, block="_round4", n=7)
    X = inception_block_b(X, block="_round5", n=7)

    # Inception C Block
    X = inception_block_c(X, block="_round1", strides_bool=True)
    X = inception_block_c(X, block="_round2")

    # Final Pooling and Dense Layer
    X = AveragePooling2D((8, 8))(X) # could this be replaced by 1x8 and 8x1 pooling?
    #   X = AveragePooling2D((1, 8))(X)
    #   X = AveragePooling2D((8, 1))(X)
    
    X = Flatten()(X) # if dimension before flatten is 1x1x2048 than flatten can be removed
    X = Dense(classes, activation="softmax", name="inception_v3_fc", kernel_initializer=glorot_uniform(seed=0))(X)

    # Create Model
    model = Model(inputs=X_input, outputs=X, name="Inception_V3")

    return model

In [0]:
model = inception_v3()

In [9]:
model.summary()

Model: "Inception_V3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv1a (Conv2D)                 (None, 149, 149, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv1b (Conv2D)                 (None, 147, 147, 32) 9248        conv1a[0][0]                     
__________________________________________________________________________________________________
conv1c (Conv2D)                 (None, 147, 147, 64) 18496       conv1b[0][0]                     
_______________________________________________________________________________________